In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
%load_ext rpy2.ipython

In [3]:
%R library(ggplot2)

R[write to console]: Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang



array(['ggplot2', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'], dtype='<U9')

In [4]:
#function to make df readable into R
def sametype(df):
    cols = df.columns
    for index, val in enumerate(cols):
        if df[val].dtype == 'O':
            df[val] = df[val].astype('str')
    return df

attendance analysis
---

In [9]:
#read DATA in

#first read in attendance data
attendance_df = pd.read_csv('../data/historical/HomeGames.csv')

#read in franchises for active or not
franchises_df = pd.read_csv('../data/historical/TeamsFranchises.csv')

#park info
parks_df = pd.read_csv('../data/historical/Parks.csv')

#team info
teams_df = pd.read_csv('../data/historical/Teams.csv')

#postseason stats
post_season_df = pd.read_csv('../data/historical/SeriesPost.csv')

#stadium history


In [10]:
active_franchises_df = franchises_df.loc[franchises_df.active == 'Y']

In [11]:
#teams_df.head()

In [12]:
#goes from 2895 rows to 2656 when filtering for only active
#also filter for last 50 years, from year 1969 and number of rows is 1398
active_teams_df = teams_df.loc[(teams_df.franchID.isin(active_franchises_df.franchID)) & (teams_df.yearID > 1968)]

In [13]:
active_teams_df = active_teams_df[['yearID', 'lgID', 'teamID', 'franchID', 'Rank', 'W', 'name', 'park', 'attendance']]

In [22]:
active_teams_df.head()

,yearID,lgID,teamID,franchID,Rank,W,name,park,attendance
1517,1969,NL,ATL,ATL,1,93,Atlanta Braves,Atlanta-Fulton County Stadium,1458320.0
1518,1969,AL,BAL,BAL,1,109,Baltimore Orioles,Memorial Stadium,1062069.0
1519,1969,AL,BOS,BOS,3,87,Boston Red Sox,Fenway Park II,1833246.0
1520,1969,AL,CAL,ANA,3,71,California Angels,Anaheim Stadium,758388.0
1521,1969,AL,CHA,CHW,5,68,Chicago White Sox,Comiskey Park,589546.0


In [23]:
post_season_df[['yearID', 'round', 'teamIDwinner', 'teamIDloser']].head()

,yearID,round,teamIDwinner,teamIDloser
0,1884,WS,PRO,NY4
1,1885,WS,CHN,SL4
2,1886,WS,SL4,CHN
3,1887,WS,DTN,SL4
4,1888,WS,NY1,SL4


In [35]:
#need to see if made playoffs and worldseries that year
with_series = pd.merge(active_teams_df, post_season_df[['yearID', 'round', 'teamIDwinner']], how = 'left', right_on = ['yearID', 'teamIDwinner'], left_on = ['yearID', 'teamID'])
with_series = pd.merge(with_series, post_season_df[['yearID', 'round', 'teamIDloser']], how = 'left', right_on = ['yearID', 'teamIDloser'], left_on = ['yearID', 'teamID'])
with_series.rename(columns={'round_x':'round won', 'round_y':'round lost'}, inplace=True)
with_series = with_series.drop(['teamIDwinner', 'teamIDloser'], axis=1)
with_series

,yearID,lgID,teamID,franchID,Rank,W,name,park,attendance,round won,round lost
0,1969,NL,ATL,ATL,1,93,Atlanta Braves,Atlanta-Fulton County Stadium,1458320.0,NaN,NLCS
1,1969,AL,BAL,BAL,1,109,Baltimore Orioles,Memorial Stadium,1062069.0,ALCS,WS
2,1969,AL,BOS,BOS,3,87,Boston Red Sox,Fenway Park II,1833246.0,NaN,NaN
3,1969,AL,CAL,ANA,3,71,California Angels,Anaheim Stadium,758388.0,NaN,NaN
4,1969,AL,CHA,CHW,5,68,Chicago White Sox,Comiskey Park,589546.0,NaN,NaN
5,1969,NL,CHN,CHC,2,92,Chicago Cubs,Wrigley Field,1674993.0,NaN,NaN
6,1969,NL,CIN,CIN,3,89,Cincinnati Reds,Crosley Field,987991.0,NaN,NaN
7,1969,AL,CLE,CLE,6,62,Cleveland Indians,Cleveland Stadium,619970.0,NaN,NaN
8,1969,AL,DET,DET,2,90,Detroit Tigers,Tiger Stadium,1577481.0,NaN,NaN
9,1969,NL,HOU,HOU,5,81,Houston Astros,Astrodome,1442995.0,NaN,NaN
